In [ ]:
import os
import sys
import json

import pandas as pd
import numpy as np

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))

In [ ]:
from standalone.plant_loop import PlantLoop
import glhe

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
d = glhe.globals.functions.load_json('EWT_MFRTRT_input.json')
d['ground-heat-exchanger'][0]['g-function-path'] = join(cwd, 'EWT_experimental_g_functions.csv')
d['simulation']['time-steps-per-hour'] = 60
d['simulation']['runtime'] = 14440

In [ ]:
with open(join(cwd, 'in.json'), 'w') as f:
    f.write(json.dumps(d, sort_keys=True, indent=2, separators=(',', ': ')))

In [ ]:
PlantLoop('in.json').simulate()

In [ ]:
df = pd.read_csv('out.csv', index_col='Date/Time', parse_dates=True)

In [ ]:
h = d['borehole-definitions'][0]['depth']
alpha = d['soil']['conductivity'] / (d['soil']['density'] * d['soil']['specific-heat'])
ts = h**2 / (9*alpha)

In [ ]:
df[['GroundHeatExchanger:GHE 1:Inlet Temp. [C]',
   'GroundHeatExchanger:GHE 1:Outlet Temp [C]']].plot(grid=True)

In [ ]:
plt.semilogx(df['Elapsed Time [s]'].values / ts, df['GroundHeatExchanger:GHE 1:Inlet Temp. [C]'].values, label='Inlet')
plt.semilogx(df['Elapsed Time [s]'].values / ts, df['GroundHeatExchanger:GHE 1:Outlet Temp [C]'].values, label='Outlet')
plt.ylabel('Temperature [C]')
plt.xlabel('t/ts')
plt.xlim(1e-8, 1e-3)
plt.legend()
plt.show()